# Quick Insights on How to Advertise More Effectively by Using Travel Pony Data from Facebook
## by Sophie Li
### APRD6342 Python Exercise 4

In [10]:
import csv
import pandas as pd 
import numpy as np 
import arrow
import statsmodels.api as sm

In [2]:
csvfile = 'Travel Pony Facebook.csv'
tpf = pd.read_csv(csvfile)
print(tpf.head(2))

  Start Date End Date  Campaign Name   Reach  Frequency  Impressions  Clicks  \
0    12/2/13  12/2/13  Cyber Weekend  278117   2.314303       643647     336   
1    12/1/13  12/1/13  Cyber Weekend  221255   2.079090       460009     376   

   Unique Clicks  Amount Spent (USD)  Page Likes  Page Engagement  \
0            313               300.0           3              246   
1            348               300.0           6              215   

   Post Engagement  Post Likes  Post Comments  Post Shares  Photo Views  \
0              243           0              0            0            0   
1              209           0              0            0            0   

   Website Clicks  Cost per Page Engagement (USD)  
0             243                        1.219512  
1             209                        1.395349  


### First, create the analytics 'cost per impression' by dividing Amount Spent / Impressions. 

### When considering 'cost per impression', what day of the week works best (cheapest to generate impressions)? 
### What day works worst (most expensive)?

### A: Friday - most expensive; Saturday - cheapest to generate impressions. 

In [6]:
tpf['Cost per Impression'] = tpf['Amount Spent (USD)']/tpf['Impressions']

dayofweek = []

for row in range(len(tpf)):
    date = arrow.get(tpf['Start Date'][row], 'M/D/YY')
    dayofweek.append(date.isoweekday())

tpf['Day'] = dayofweek

group = tpf.groupby('Day')
group.aggregate(np.mean)

,Reach,Frequency,Impressions,Clicks,Unique Clicks,Amount Spent (USD),Page Likes,Page Engagement,Post Engagement,Post Likes,Post Comments,Post Shares,Photo Views,Website Clicks,Cost per Page Engagement (USD),Cost per Impression
Day,,,,,,,,,,,,,,,,
1,1842.864177,1.017603,2525.074703,7.536503,6.521222,7.427097,0.670628,7.634975,6.964346,1.095076,0.040747,0.225806,2.994907,2.607810,0.492924,0.002910
2,1889.215190,1.012193,1926.738397,9.419831,8.151899,9.455084,1.094937,9.679325,8.584388,1.280591,0.065401,0.202532,3.877637,3.158228,0.592907,0.003040
3,1494.235195,1.009059,1516.478849,7.781726,6.524535,7.956024,0.934010,8.131980,7.197970,1.241963,0.067682,0.187817,3.225042,2.475465,0.539237,0.002981
4,1413.081882,1.011668,1449.837979,8.054007,6.980836,7.640209,0.855401,8.634146,7.778746,1.371080,0.094077,0.200348,4.012195,2.101045,0.560598,0.003398
5,2238.474104,1.012024,3037.984064,10.147410,8.848606,10.014741,0.844622,10.635458,9.790837,1.424303,0.051793,0.250996,4.745020,3.318725,0.476606,0.004097
6,2281.755511,1.014517,3152.184369,10.014028,8.857715,9.625511,0.923848,9.573146,8.649299,1.102204,0.048096,0.212425,3.595190,3.691383,0.415395,0.002629
7,2203.596639,1.016073,2775.644958,9.981092,8.808824,9.676534,0.831933,8.794118,7.960084,0.962185,0.029412,0.218487,3.220588,3.529412,0.491080,0.003688


### Next, which correlation is the strongest? 
### Amount Spent vs. Reach, Frequency, Unique Clicks, Page Likes

In [8]:
AS_R = np.corrcoef(tpf['Amount Spent (USD)'], tpf['Reach'])
AS_F = np.corrcoef(tpf['Amount Spent (USD)'], tpf['Frequency'])
AS_UC = np.corrcoef(tpf['Amount Spent (USD)'], tpf['Unique Clicks'])
AS_PL = np.corrcoef(tpf['Amount Spent (USD)'], tpf['Page Likes'])

aList = [AS_R, AS_F, AS_UC, AS_PL]
print (aList) 

[array([[1.        , 0.70312381],
       [0.70312381, 1.        ]]), array([[1.        , 0.13020087],
       [0.13020087, 1.        ]]), array([[1.        , 0.88299318],
       [0.88299318, 1.        ]]), array([[1.        , 0.75761193],
       [0.75761193, 1.        ]])]


### A: Amount Spent vs. Unique Clicks has the strongest correlation (0.88). It means that among the four variables, the amount spent will increase more when we get more unique clicks. 

### Finally, perform a simple multiple regression analysis where Unique Clicks is the dependent variable, and Reach and Frequency are the independent (predictor) variables.
### What variable most strongly predicts unique clicks?

In [11]:
x = tpf[['Reach', 'Frequency']]
y = tpf['Unique Clicks']

model = sm.OLS(y, x).fit()
predictions = model.predict(x)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Unique Clicks   R-squared:                       0.557
Model:                            OLS   Adj. R-squared:                  0.556
Method:                 Least Squares   F-statistic:                     2325.
Date:                Thu, 18 Oct 2018   Prob (F-statistic):               0.00
Time:                        16:11:26   Log-Likelihood:                -15973.
No. Observations:                3705   AIC:                         3.195e+04
Df Residuals:                    3703   BIC:                         3.196e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Reach          0.0019   3.12e-05     62.490      0.000       0.002       0.002
Frequency      3.6139      0.298     12.109      0.000       3.029       4.199
==============================================================================
Omnibus:                     5107.616   Durbin-Watson:                   0.803
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          5128392.747
Skew:                           7.331   Prob(JB):                         0.00
Kurtosis:                     184.674   Cond. No.                     9.84e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.84e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### From the summary, Frequency most strongly predicts unique clicks. It means that Frequency is more correlated to Unique Clicks because of the larger coefficient. 